# Sample for Tekton Pipeline Python SDK

This is a sample for Tekton Pipeline Python SDK.

The notebook shows how to use Tekton Pipeline Python SDK to create, get and delete TaskRun.

### Ensure the Tekton Pipeline Python SDK is installed

The Python SDK already published in https://pypi.org/project/tekton-pipeline/

In [1]:
!pip show tekton-pipeline

Name: tekton-pipeline
Version: 0.1.3
Summary: Tekton Pipeline Python SDK
Home-page: https://github.com/tektoncd/pipeline.git
Author: Tekton Authors
Author-email: hejinchi@cn.ibm.com
License: Apache License Version 2.0
Location: /opt/python/python37/lib/python3.7/site-packages
Requires: setuptools, python-dateutil, table-logger, certifi, kubernetes, six, urllib3
Required-by: 


### Instantiate a Tekton Python Client

In [2]:
from tekton_pipeline import TektonClient

tekton_client = TektonClient()

### Define a Task and submit

In [3]:
from kubernetes import client as k8s_client
from tekton_pipeline import V1beta1Task
from tekton_pipeline import V1beta1TaskSpec
from tekton_pipeline import V1beta1Step

# Define the task
task = V1beta1Task(api_version='tekton.dev/v1beta1',
                   kind='TaskRun',
                   metadata=k8s_client.V1ObjectMeta(name='sdk-sample-task'),
                   spec=V1beta1TaskSpec(
                       steps=[V1beta1Step(name='default',
                              image='ubuntu',
                              script='sleep 30;echo "This is a sdk demo."')]
                   ))

# Submit the task to cluster
tekton_client.create(entity='task', body=task, namespace='default')

{'apiVersion': 'tekton.dev/v1beta1',
 'kind': 'Task',
 'metadata': {'creationTimestamp': '2020-11-05T09:05:06Z',
  'generation': 1,
  'name': 'sdk-sample-task',
  'namespace': 'default',
  'resourceVersion': '6266273',
  'selfLink': '/apis/tekton.dev/v1beta1/namespaces/default/tasks/sdk-sample-task',
  'uid': '281eeeca-043e-4a27-9243-0b3202c3ace1'},
 'spec': {'steps': [{'image': 'ubuntu',
    'name': 'default',
    'resources': {},
    'script': 'sleep 30;echo "This is a sdk demo."'}]}}

In [4]:
# Update the task defination
task = V1beta1Task(api_version='tekton.dev/v1beta1',
                   kind='TaskRun',
                   metadata=k8s_client.V1ObjectMeta(name='sdk-sample-task'),
                   spec=V1beta1TaskSpec(
                       steps=[V1beta1Step(name='default',
                              image='ubuntu',
                              script='sleep 30;echo "This is a sdk patch demo."')]
                   ))

# Patch the task
tekton_client.patch(entity='task', name='sdk-sample-task', body=task, namespace='default')

{'apiVersion': 'tekton.dev/v1beta1',
 'kind': 'Task',
 'metadata': {'creationTimestamp': '2020-11-05T09:05:06Z',
  'generation': 2,
  'name': 'sdk-sample-task',
  'namespace': 'default',
  'resourceVersion': '6266302',
  'selfLink': '/apis/tekton.dev/v1beta1/namespaces/default/tasks/sdk-sample-task',
  'uid': '281eeeca-043e-4a27-9243-0b3202c3ace1'},
 'spec': {'steps': [{'image': 'ubuntu',
    'name': 'default',
    'resources': {},
    'script': 'sleep 30;echo "This is a sdk patch demo."'}]}}

### Get the Patched Task

In [5]:
tekton_client.get(entity='task', name='sdk-sample-task', namespace='default')

{'apiVersion': 'tekton.dev/v1beta1',
 'kind': 'Task',
 'metadata': {'creationTimestamp': '2020-11-05T09:05:06Z',
  'generation': 2,
  'name': 'sdk-sample-task',
  'namespace': 'default',
  'resourceVersion': '6266302',
  'selfLink': '/apis/tekton.dev/v1beta1/namespaces/default/tasks/sdk-sample-task',
  'uid': '281eeeca-043e-4a27-9243-0b3202c3ace1'},
 'spec': {'steps': [{'image': 'ubuntu',
    'name': 'default',
    'resources': {},
    'script': 'sleep 30;echo "This is a sdk patch demo."'}]}}

### Define a TaskRun

In [6]:
from tekton_pipeline import V1beta1TaskRun
from tekton_pipeline import V1beta1TaskRunSpec
from tekton_pipeline import V1beta1TaskRef

# Define the taskrun
taskrun = V1beta1TaskRun(
    api_version='tekton.dev/v1beta1',
    kind='TaskRun',
    metadata=k8s_client.V1ObjectMeta(name='sdk-sample-taskrun'),
    spec=V1beta1TaskRunSpec(
        task_ref=V1beta1TaskRef(name='sdk-sample-task')
    ))

# Create the taskrun
tekton_client.create(entity='taskrun', body=taskrun, namespace='default')

{'apiVersion': 'tekton.dev/v1beta1',
 'kind': 'TaskRun',
 'metadata': {'creationTimestamp': '2020-11-05T09:05:13Z',
  'generation': 1,
  'labels': {'app.kubernetes.io/managed-by': 'tekton-pipelines'},
  'name': 'sdk-sample-taskrun',
  'namespace': 'default',
  'resourceVersion': '6266343',
  'selfLink': '/apis/tekton.dev/v1beta1/namespaces/default/taskruns/sdk-sample-taskrun',
  'uid': '43e51f39-749c-4ac9-b3f9-6f3cb5df0180'},
 'spec': {'serviceAccountName': 'default',
  'taskRef': {'kind': 'Task', 'name': 'sdk-sample-task'},
  'timeout': '1h0m0s'}}

### Watch the created TaskRun

In [7]:
tekton_client.get(entity='taskrun', name='sdk-sample-taskrun', namespace='default', watch=True)

NAME                 SUCCEEDED            REASON               STARTED              COMPLETED           
sdk-sample-taskrun   Unknown              Pending              2020-11-05T09:05:13Z                     
sdk-sample-taskrun   Unknown              Pending              2020-11-05T09:05:13Z                     
sdk-sample-taskrun   Unknown              Pending              2020-11-05T09:05:13Z                     
sdk-sample-taskrun   Unknown              Running              2020-11-05T09:05:13Z                     
sdk-sample-taskrun   True                 Succeeded            2020-11-05T09:05:13Z 2020-11-05T09:05:54Z


### Define Pipeline and submit 

In [8]:
from tekton_pipeline import V1beta1Pipeline
from tekton_pipeline import V1beta1PipelineSpec
from tekton_pipeline import V1beta1PipelineTask

# Define the pipeline
pipeline = V1beta1Pipeline(
    api_version='tekton.dev/v1beta1',
    kind='Pipeline',
    metadata=k8s_client.V1ObjectMeta(name='sdk-sample-pipeline'),
    spec=V1beta1PipelineSpec(tasks=[
        V1beta1PipelineTask(
            name='sdk-sample-pipeline-task',
            task_ref=V1beta1TaskRef(name='sdk-sample-task')
        )]
    ))


# Create the pipeline
tekton_client.create(entity='pipeline', body=pipeline, namespace='default')

{'apiVersion': 'tekton.dev/v1beta1',
 'kind': 'Pipeline',
 'metadata': {'creationTimestamp': '2020-11-05T09:07:18Z',
  'generation': 1,
  'name': 'sdk-sample-pipeline',
  'namespace': 'default',
  'resourceVersion': '6267499',
  'selfLink': '/apis/tekton.dev/v1beta1/namespaces/default/pipelines/sdk-sample-pipeline',
  'uid': 'b4e267a2-e7a4-4d88-bb3e-b4fdda0acca0'},
 'spec': {'tasks': [{'name': 'sdk-sample-pipeline-task',
    'taskRef': {'kind': 'Task', 'name': 'sdk-sample-task'}}]}}

### Define PipelineRun and submit

In [9]:
from tekton_pipeline import V1beta1PipelineRun
from tekton_pipeline import V1beta1PipelineRunSpec
from tekton_pipeline import V1beta1PipelineRef

pipelinerun = V1beta1PipelineRun(
    api_version='tekton.dev/v1beta1',
    kind='PipelineRun',
    metadata=k8s_client.V1ObjectMeta(name='sdk-sample-pipelinerun'),
    spec=V1beta1PipelineRunSpec(
        pipeline_ref=V1beta1PipelineRef(
            name='sdk-sample-pipeline')
    ))


tekton_client.create(entity='pipelinerun', body=pipelinerun, namespace='default')

{'apiVersion': 'tekton.dev/v1beta1',
 'kind': 'PipelineRun',
 'metadata': {'creationTimestamp': '2020-11-05T09:07:23Z',
  'generation': 1,
  'name': 'sdk-sample-pipelinerun',
  'namespace': 'default',
  'resourceVersion': '6267538',
  'selfLink': '/apis/tekton.dev/v1beta1/namespaces/default/pipelineruns/sdk-sample-pipelinerun',
  'uid': '47d1e953-f1a2-492b-b1ea-b6f004fd260e'},
 'spec': {'pipelineRef': {'name': 'sdk-sample-pipeline'},
  'serviceAccountName': 'default',
  'timeout': '1h0m0s'}}

### Watch the PipelineRun

In [10]:
tekton_client.get(entity='pipelinerun', name='sdk-sample-pipelinerun', namespace='default', watch=True)

NAME                 SUCCEEDED            REASON               STARTED              COMPLETED           
sdk-sample-pipeli... Unknown              Running              2020-11-05T09:07:23Z                     
sdk-sample-pipeli... Unknown              Running              2020-11-05T09:07:23Z                     
sdk-sample-pipeli... Unknown              Running              2020-11-05T09:07:23Z                     
sdk-sample-pipeli... Unknown              Running              2020-11-05T09:07:23Z                     
sdk-sample-pipeli... True                 Succeeded            2020-11-05T09:07:23Z 2020-11-05T09:08:03Z


### Delete a TaskRun

In [11]:
tekton_client.delete(entity='pipelinerun', name='sdk-sample-pipelinerun', namespace='default')

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'sdk-sample-pipelinerun',
  'group': 'tekton.dev',
  'kind': 'pipelineruns',
  'uid': '47d1e953-f1a2-492b-b1ea-b6f004fd260e'}}

In [12]:
tekton_client.delete(entity='pipeline', name='sdk-sample-pipeline', namespace='default')

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'sdk-sample-pipeline',
  'group': 'tekton.dev',
  'kind': 'pipelines',
  'uid': 'b4e267a2-e7a4-4d88-bb3e-b4fdda0acca0'}}

In [13]:
tekton_client.delete(entity='taskrun', name='sdk-sample-taskrun', namespace='default')

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'sdk-sample-taskrun',
  'group': 'tekton.dev',
  'kind': 'taskruns',
  'uid': '43e51f39-749c-4ac9-b3f9-6f3cb5df0180'}}

In [14]:
tekton_client.delete(entity='task', name='sdk-sample-task', namespace='default')

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'sdk-sample-task',
  'group': 'tekton.dev',
  'kind': 'tasks',
  'uid': '281eeeca-043e-4a27-9243-0b3202c3ace1'}}